<a href="https://colab.research.google.com/github/renato-penna/fiap-hackaton-fase05/blob/main/train_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 33.7 MB/s eta 0:00:00


In [ ]:
from ultralytics import YOLO
from google.colab import drive
from pathlib import Path
import shutil
import os

# =========================================================
# 0. Mount Google Drive
# =========================================================
drive.mount('/content/drive')

DRIVE_ROOT = "/content/drive/MyDrive/colab"
PROJECT_NAME = "cloud-arch-security-mvp"

DRIVE_PROJECT = f"{DRIVE_ROOT}/{PROJECT_NAME}"
DRIVE_DATASET = f"{DRIVE_PROJECT}/dataset"
# Não precisamos copiar a pasta models do drive se formos baixar o peso da internet
# mas se você tiver pesos customizados lá, mantenha.

CONTENT_PROJECT = "/content/yolo-project"
CONTENT_DATASET = f"{CONTENT_PROJECT}/dataset"

# =========================================================
# 1. Preparar ambiente rápido (/content)
# =========================================================
# Limpa execuções anteriores no ambiente local para não lotar o disco
if Path(CONTENT_PROJECT).exists():
    shutil.rmtree(CONTENT_PROJECT)

os.makedirs(CONTENT_PROJECT, exist_ok=True)

print("⏳ Copiando dataset para disco local (SSD)... isso acelera o treino...")
shutil.copytree(DRIVE_DATASET, CONTENT_DATASET)
print("✅ Dataset copiado!")

# Trabalhar sempre em /content
%cd /content/yolo-project

# =========================================================
# 2. Carregar modelo (AQUI ESTÁ A GRANDE MUDANÇA)
# =========================================================
# Usamos o 'yolo11m.pt' (Medium). Ele é baixado automaticamente se não existir.
# O 'Nano' (n) é muito fraco para diferenciar ícones complexos.
model = YOLO("yolo11m.pt")

# =========================================================
# 3. Treinar modelo (Configuração Otimizada para T4)
# =========================================================
results = model.train(
    data="dataset/data.yaml",

    # TREINAMENTO
    epochs=150,       # Aumentado para dar tempo de aprender
    patience=20,      # Para se não melhorar após 20 épocas (Early Stopping)
    batch=16,         # 16 é seguro para T4 com modelo Medium. Se der erro de memória, baixe para 8.
    imgsz=800,        # Tamanho padrão. Se os ícones forem muito pequenos, tente 800 (vai ficar mais lento)
    save_period=10,   # salva last.pt a cada 10 epochs

    # OTIMIZAÇÃO
    cache=False,      # Mantém imagens na RAM (acelera muito)
    workers=2,        # Colab geralmente aguenta 8 workers para carregar dados
    exist_ok=True,    # Permite sobrescrever pasta project sem erro

    # VISUALIZAÇÃO
    plots=True,       # Gera gráficos de perda/precisão
    name='train_mvp_medium', # Nome da pasta interna

)

# =========================================================
# 4. Salvar resultados no Drive IMEDIATAMENTE
# =========================================================
# O Colab pode desconectar. Vamos garantir que o 'best.pt' vá para o Drive.

DRIVE_RUNS = f"{DRIVE_PROJECT}/runs"
SOURCE_WEIGHTS = f"runs/detect/train_mvp_medium/weights"
DEST_WEIGHTS = f"{DRIVE_PROJECT}/weights_backup"

# Cria pasta de backup no drive se não existir
os.makedirs(DEST_WEIGHTS, exist_ok=True)

print("💾 Salvando o melhor modelo no Google Drive...")

# Copia apenas os pesos (o mais importante) para garantir
if os.path.exists(f"{SOURCE_WEIGHTS}/best.pt"):
    shutil.copy(f"{SOURCE_WEIGHTS}/best.pt", f"{DEST_WEIGHTS}/best_medium.pt")
    shutil.copy(f"{SOURCE_WEIGHTS}/last.pt", f"{DEST_WEIGHTS}/last_medium.pt")
    print(f"✅ SUCESSO! Modelo salvo em: {DEST_WEIGHTS}/best_medium.pt")
else:
    print("⚠️ Algo deu errado, arquivo best.pt não encontrado.")

# Copia a pasta runs inteira (logs, gráficos) se quiser
# shutil.copytree("runs", DRIVE_RUNS, dirs_exist_ok=True)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
⏳ Copiando dataset para disco local (SSD)... isso acelera o treino...
✅ Dataset copiado!
/content/yolo-project
Ultralytics 8.4.7 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=dataset/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False,

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!nvidia-smi


Mon Jan 26 16:55:05 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   57C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----